In [12]:
import skimage
from skimage import data, util
import nibabel as nib
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import scipy.misc


from skimage.feature import hog
from skimage import data, exposure
import numpy as np
from skimage.measure import label

import os


In [13]:
a=[]
root_dir_train = "/home/mmlab/Sesh/Sesh_Valid"
for mri_file in os.listdir(root_dir_train):
    a.append(int(mri_file))
#sorted(a)
x=sorted(a)
print(len(x))


40


In [14]:
for i in range(len(x)):
#for i in range(27,50):
    img_path=root_dir_train+'/'+str(x[i])
    #print(img_path)
    for mri_file in os.listdir(img_path):
        mri_file = img_path+'/'+mri_file
        if((mri_file[-20:-17])=='seg'):
            print(mri_file)
            _img = nib.load(mri_file).get_data()
        
            _img = _img.transpose(2,0,1)

            img = np.zeros((138,186,186))

            a0 = _img.shape[0]  
            a1 = _img.shape[2]  
        
            a0 = (a0 - 138)//2
            a1 = (a1 - 186)//2
        
            img = _img[a0:a0+138, a1:a1+186, a1:a1+186]
            label_img = label(img, connectivity=img.ndim)
            props = skimage.measure.regionprops(label_img)

            Centroid = props[0]['Centroid']
            MajorAxisLength = props[0]['major_axis_length']
            if((mri_file[-27:-24])=='074'):
                MinorAxisLength = MajorAxisLength
            else:
                MinorAxisLength = props[0]['minor_axis_length']
            

            BBox = props[0]['bbox']
            DiagonalAxis = []
            DiagonalAxis.append(BBox[3]-BBox[0])
            DiagonalAxis.append(BBox[4]-BBox[1])
            DiagonalAxis.append(BBox[5]-BBox[2])
            Dummy = DiagonalAxis
            Denominator = np.power((DiagonalAxis[0]),2)+np.power((DiagonalAxis[1]),2)+np.power((DiagonalAxis[2]),2)
            Denominator = np.sqrt(Denominator)
            DiagonalAxis = DiagonalAxis/Denominator


            DiagonalPerp = []
            DiagonalPerp.append(1)
            DiagonalPerp.append(1)
            DiagonalPerp.append(-(Dummy[0]*1 +Dummy[1]*1)/Dummy[2])
            Denominator = np.power((DiagonalPerp[0]),2)+np.power((DiagonalPerp[1]),2)+np.power((DiagonalPerp[2]),2)
            Denominator = np.sqrt(Denominator)
            DiagonalPerp = DiagonalPerp/Denominator


            Extent = props[0]['extent']
            Diameter = props[0]['equivalent_diameter']
            EigenValues = props[0]['inertia_tensor_eigvals']
            Solidity = props[0]['solidity']
            Vectors = props[0]['inertia_tensor']
            FirstAxis = Vectors[0]
            Denominator = np.power((Vectors[0][0]),2)+np.power((Vectors[0][1]),2)+ np.power((Vectors[0][2]),2)
            Denominator = np.sqrt(Denominator)
            FirstAxisLength = Denominator
            FirstAxis = FirstAxis/Denominator

            SecondAxis = Vectors[1]
            Denominator = np.power((Vectors[1][0]),2)+np.power((Vectors[1][1]),2)+ np.power((Vectors[1][2]),2)
            Denominator = np.sqrt(Denominator)
            SecondAxisLength = Denominator
            SecondAxis = SecondAxis/Denominator

            ThirdAxis = Vectors[2]
            Denominator = np.power((Vectors[2][0]),2)+np.power((Vectors[2][1]),2)+ np.power((Vectors[2][2]),2)
            Denominator = np.sqrt(Denominator)
            ThirdAxisLength = Denominator
            ThirdAxis = ThirdAxis/Denominator
            kurt = (kurtosis(kurtosis(kurtosis(img, fisher=True))))
            histo=0
            i=0
            for i in range(img.shape[0]):
                fd, hog_image = hog(img[i,:,:], orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=True, multichannel=False)
                #print(np.sum(hog_image))
                histo = histo+np.sum((hog_image))
            histo = histo/1000
            hemorrhage  = np.sum(np.sum(np.sum(img)))
            hemorrhage = hemorrhage/135442
            
            def fractal_dimension(Z, threshold=0.9):

                def boxcount(Z, k):
                    S = np.add.reduceat(
                        np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                                           np.arange(0, Z.shape[1], k), axis=1)

                    return len(np.where((S > 0) & (S < k*k))[0])


                Z = (Z < threshold)


                p = min(Z.shape)

                n = 2**np.floor(np.log(p)/np.log(2))

                n = int(np.log(n)/np.log(2))

                sizes = 2**np.arange(n, 1, -1)

                counts = []
                for size in sizes:
                    counts.append(boxcount(Z, size))

                coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
                return -coeffs[0]

            FractalDim = fractal_dimension(img)
            Entropy = skimage.measure.shannon_entropy(img, base=2)
            parameters = []
            parameters.append(Centroid)
            parameters.append(MajorAxisLength)
            parameters.append(MinorAxisLength)
            parameters.append(DiagonalAxis)
            parameters.append(DiagonalPerp)
            parameters.append(Extent)
            parameters.append(Diameter)
            parameters.append(EigenValues)
            parameters.append(Solidity)
            parameters.append(FirstAxis)
            parameters.append(SecondAxis)
            parameters.append(ThirdAxis)
            parameters.append(FirstAxisLength)
            parameters.append(SecondAxisLength)
            parameters.append(ThirdAxisLength)
            parameters.append(kurt)
            parameters.append(histo)
            parameters.append(hemorrhage)
            parameters.append(FractalDim)
            parameters.append(Entropy)
            parameters = np.asarray(parameters)
            np.save(mri_file[-27:-21]+"_seg.npy",parameters)

/home/mmlab/Sesh/Sesh_Valid/0/001ct1_seg_resampled.nii.gz


/usr/local/lib/python3.5/dist-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


/home/mmlab/Sesh/Sesh_Valid/1/001ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/2/095ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/3/095ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/4/096ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/5/096ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/6/098ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/7/098ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/8/101ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/9/101ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/12/107ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/13/107ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/14/109ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/15/109ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/16/111ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/17/111ct2_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/22/013ct1_seg_resampled.nii.gz
/home/mmlab/Sesh/Sesh_Valid/23/013ct2_seg_resampled.nii.gz
/home/

In [ ]:
#Format
how2read = []
how2read.append("Centroid,3")
how2read.append("MajorAxisLength,1")
how2read.append()